# Big Data Platform
## Assignment 3: ServerLess

**By:**  

Oren Ben-Eliyahu 204079453 
<br>Yuval Barkan, 205714447

<br><br>

**The goal of this assignment is to:**
- Understand and practice the details of Serverless

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading (not Zip):
    - Jupyter Notebook
    - 2 Log files
    - Additional local scripts
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.

  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q0 - 10 points - Setup
- Q1 - 40 points - Serverless MapReduceEngine
- Q2 - 20 points - MapReduce job to calculate inverted index
- Q3 - 30 points - Shuffle

`Total: 100`

<br><br>

# Question 0
## Setup

1. Navigate to IBM Cloud and open a trial account. No need to provide a credit card
2. Choose IBM Cloud Object Storage service from the catalog
3. Create a new bucket in IBM Cloud Object Storage
4. Create credentials for the bucket with HMAC (access key and secret key)
5. Choose IBM Cloud Functions service from the catalog and create a service


#### Lithops setup
1. By using “git” tool, install master branch of the Lithops project from
https://github.com/lithops-cloud/lithops
2. Follow Lithops documentation and configure Lithops against IBM Cloud Functions and IBM Cloud Object Storage
3. Configure Lithops log level to be in DEBUG mode
4. Run Hello World example by using Futures API and verify all is working properly.


#### IBM Cloud Object Storage setup
1. Upload all the input CSV files that you used in homework 2 into the bucket you created in IBM Cloud Object Storage


<br><br><br>

In [1]:
!pip install git+https://github.com/lithops-cloud/lithops.git

  Cloning https://github.com/lithops-cloud/lithops.git to /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/pip-req-build-nwzj8cr4


  Created wheel for lithops: filename=lithops-2.5.9.dev0-py3-none-any.whl size=313591 sha256=c029296939bcac64fc9f53b064fc313822b6738c000c4a12f37dccedacc136ab
  Stored in directory: /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/pip-ephem-wheel-cache-bnabjdnd/wheels/f2/7a/47/12c1c1acee8bccb6b8e4b06ee26d13fe6f304831832bb2a33d
Successfully built lithops


In [2]:
!lithops test

2022-01-05 19:31:13,929 [INFO] lithops.config -- Lithops v2.5.9.dev0
2022-01-05 19:31:13,931 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-01-05 19:31:13,931 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-01-05 19:31:13,931 [INFO] lithops.invokers -- ExecutorID a7b1e6-0 | JobID A000 - Selected Runtime: python 
2022-01-05 19:31:13,933 [INFO] lithops.invokers -- ExecutorID a7b1e6-0 | JobID A000 - Starting function invocation: hello() - Total: 1 activations
2022-01-05 19:31:14,260 [INFO] lithops.invokers -- ExecutorID a7b1e6-0 | JobID A000 - View execution logs at /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/lithops/logs/a7b1e6-0-A000.log
2022-01-05 19:31:14,260 [INFO] lithops.wait -- ExecutorID a7b1e6-0 - Getting results from functions

  100%|██████████████████████████████████████████████████████████████████| 1/1  

2022-01-05 19:31:16,301 [INFO] lithops.executors -- ExecutorID a7b1e6-0 - Cleaning te

In [3]:
!pip install lithops aws

In [4]:
!pip install boto3

In [102]:
import lithops
import os
import boto3
from io import StringIO
import pandas as pd

In [96]:
AWSAccessKeyId = os.environ.get("AWSAccessKeyId")
AWSSecretKey = os.environ.get("AWSSecretKey")

In [105]:
config = {'lithops': {'backend': 'aws_lambda', 'storage': 'aws_s3', 'log_level':'DEBUG'},

          'aws':  {'access_key_id': AWSAccessKeyId,
                   'secret_access_key': AWSSecretKey},

          'aws_s3': {'storage_bucket': 'idc-big-data',
                     'region_name': 'eu-central-1'},
          
          'aws_lambda': {'execution_role': 'arn:aws:iam::981299761374:role/lithops-execution-role',
                     'region_name': 'eu-central-1'}}

In [98]:
def hello(name):
    return 'Hello {}!'.format(name)

with lithops.FunctionExecutor(config=config) as fexec:
    fut = fexec.call_async(hello, 'World')
    print(fut.result())

2022-01-07 13:02:21,396 [INFO] lithops.config -- Lithops v2.5.9.dev0
2022-01-07 13:02:21,404 [DEBUG] lithops.config -- Loading Serverless backend module: aws_lambda
2022-01-07 13:02:21,416 [DEBUG] lithops.config -- Loading Storage backend module: aws_s3
2022-01-07 13:02:21,418 [DEBUG] lithops.storage.backends.aws_s3.aws_s3 -- Creating S3 client
2022-01-07 13:02:21,459 [INFO] lithops.storage.backends.aws_s3.aws_s3 -- S3 client created - Region: eu-central-1
2022-01-07 13:02:21,460 [DEBUG] lithops.serverless.backends.aws_lambda.aws_lambda -- Creating AWS Lambda client
2022-01-07 13:02:21,462 [DEBUG] lithops.serverless.backends.aws_lambda.aws_lambda -- Creating Boto3 AWS Session and Lambda Client
2022-01-07 13:02:22,333 [INFO] lithops.serverless.backends.aws_lambda.aws_lambda -- AWS Lambda client created - Region: eu-central-1
2022-01-07 13:02:22,336 [DEBUG] lithops.invokers -- ExecutorID 1f1011-11 - Invoker initialized. Max workers: 1000
2022-01-07 13:02:22,337 [DEBUG] lithops.invokers -

Hello World!


2022-01-07 13:02:28,365 [DEBUG] lithops.invokers -- ExecutorID 1f1011-11 - Async invoker 1 finished
2022-01-07 13:02:28,365 [DEBUG] lithops.invokers -- ExecutorID 1f1011-11 - Async invoker 0 finished


# Question 1
## Serverless MapReduceEngine

Modify MapReduceEngine from homework 2 into the MapReduceServerlessEngine where map and reduce tasks executed as a serverless actions, instead of local threads. In particular:
1. Deploy all map tasks as a serverless actions by using Lithops against IBM Cloud Functions.
2. Collect results from all map tasks and store them in the same SQLite as you used in MapReduceEngine and use the same code for the sort and shuffle phase.
3. Deploy reduce tasks by using Lithops against IBM Cloud Functions. Instead of persisting results from reduce tasks, return results back to the MapReduceServerlessEngine and proceed with the same workflow as in MapReduceEngine
4. Return results of reduce tasks to the user

**Please attach:**  
Text file with all log messages Lithops printed to console during the execution. Make
sure log level is set to DEBUG mode.

#### Code:

In [103]:
import sqlite3
from sqlite3 import Error

In [99]:
def create_connection(db_data):
    try:
        conn = sqlite3.connect(db_data)
        print("Establish connection")
    except Error as e:
        print(e)
    finally:
        return conn


def create_table(conn, create_table_query):
    try:
        c = conn.cursor()
        c.execute(create_table_query)
    except Error as e:
        print(e)
        
def query(conn, select_query):
    try:
        c = conn.cursor()
        select_all = select_query
        rows = c.execute(select_all).fetchall()
        # Output to the console screen
        return rows
    
    except Error as e:
        print(e)

In [100]:
MYDATA_DB = 'temp_results.db'

create_table_temp_results = '''CREATE TABLE IF NOT EXISTS temp_results(
                                key TEXT,
                                value TEXT)
                            '''
conn = create_connection(MYDATA_DB)
create_table(conn,create_table_temp_results)

Establish connection


In [289]:
import concurrent.futures as cf

class MapReduceServerlessEngine():
    
    def execute(self, input_data, map_function, reduce_function, params):
        
        fexec = lithops.FunctionExecutor(log_level='DEBUG')
                
        # map function
        map_collector = []
        for count, csv_file in enumerate(input_data):
            fexec.call_async(inverted_map, [csv_file,params])
            map_collector.append(fexec.get_result())

        # Insert data into temp_result db:
        insert_query = '''INSERT INTO temp_results (key,value) VALUES (?,?)'''
        cur = conn.cursor()
        
        for file in map_collector:
            cur.executemany(insert_query,file)
            
        cur.close()
        
        # SQL statements:
        grouping_query = "SELECT key, GROUP_CONCAT(value,',') FROM temp_results GROUP BY key"
        reduce_input_values = query(conn,grouping_query)
        
        unique_keys = "SELECT  count(distinct key) FROM temp_results"
        unique_keys = query(conn,unique_keys)[0][0]
        
        reduce_collector =[]
        # reduce function
        for i in range(unique_keys):
            key = reduce_input_values[i][0]
            values = reduce_input_values[i][1].split(',')
            
            fexec.call_async(reduce_function, [key,values])
            reduce_collector.append(fexec.get_result())
            
        return reduce_collector

In [290]:
from os import listdir

def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [291]:
input_data = find_csv_filenames('csv_files/')

In [292]:
#input_data

In [293]:
def inverted_map(document_name_index):
    csv_dir = f'csv_files/{document_name_index[0]}'
    file = pd.read_csv(csv_dir)
    list_mycsv=[]
    for value in file.iloc[:,document_name_index[1]['column']].values:
        list_mycsv.append((value,document_name_index[0]))
    return list_mycsv

In [294]:
#inverted_map(['myCSV1.csv',params])

In [295]:
# map_collector=[]
# params = {'column':2}
# for count, csv_file in enumerate(input_data):
#     fexec = lithops.FunctionExecutor(log_level='DEBUG')
#     fexec.call_async(inverted_map, [csv_file,params])
#     map_collector.append(fexec.get_result())
# print(map_collector)
# #fexec.clean()

In [296]:
# map_collector=[]
# with lithops.FunctionExecutor(config=config) as fexec:
#     for count, csv_file in enumerate(input_data):
#         fut = fexec.map(inverted_map, [csv_file])
#         map_result = fut.get_result()
#         map_collector.append(map_result)

In [297]:
# The goal of reduce function is to create csv files. 
# Therfore we chose to optimize the process and not create unnecessary variables (such as list) and return a data frame. 

def inverted_reduce(key_documents):
    reduced_list = []
    reduced_loc = list(set(key_documents[1]))
    reduced_list.append((key_documents[0],reduced_loc))
    return reduced_list
#     df = pd.DataFrame({'key':key_documents[0],'value':list(set(key_documents[1]))})
#     return df



In [288]:
status

[[('Haifa',
   ['myCSV5.csv',
    'myCSV8.csv',
    'myCSV16.csv',
    'myCSV4.csv',
    'myCSV7.csv',
    'myCSV15.csv',
    'myCSV19.csv',
    'myCSV17.csv',
    'myCSV14.csv',
    'myCSV11.csv',
    'myCSV13.csv',
    'myCSV10.csv',
    'myCSV18.csv',
    'myCSV20.csv',
    'myCSV1.csv',
    'myCSV6.csv',
    'myCSV3.csv',
    'myCSV12.csv',
    'myCSV2.csv'])],
 [('Hamburg',
   ['myCSV18.csv',
    'myCSV20.csv',
    'myCSV11.csv',
    'myCSV9.csv',
    'myCSV14.csv',
    'myCSV8.csv',
    'myCSV12.csv',
    'myCSV7.csv',
    'myCSV6.csv',
    'myCSV17.csv',
    'myCSV16.csv',
    'myCSV5.csv',
    'myCSV10.csv',
    'myCSV13.csv',
    'myCSV19.csv'])],
 [('Kiel',
   ['myCSV11.csv',
    'myCSV3.csv',
    'myCSV7.csv',
    'myCSV2.csv',
    'myCSV4.csv',
    'myCSV20.csv',
    'myCSV10.csv',
    'myCSV14.csv',
    'myCSV19.csv',
    'myCSV16.csv',
    'myCSV15.csv',
    'myCSV9.csv',
    'myCSV1.csv',
    'myCSV12.csv',
    'myCSV5.csv'])],
 [('London',
   ['myCSV14.csv',
    'myCSV1

# Task 2
## Submit MapReduce job to calculate inverted index
1. Use input_data: `cos://bucket/<path to CSV data>`
2. Submit MapReduce job with reduce and map functions as you used in homework 2, as follows

    `mapreduce = MapReduceServerlessEngine()`  
    `results = mapreduce.execute(input_data, inverted_map, inverted_index)`   
    `print(results)`

**Please attach:**  
Text file with all log messages Lithops printed to console during the execution. Make
sure log level is set to DEBUG mode.

#### Code:

In [287]:
mapreduce = MapReduceServerlessEngine()
params = {'column':2}
results = mapreduce.execute(input_data, inverted_map, inverted_reduce, params)
print(results)

2022-01-07 15:34:30,113 [INFO] lithops.config -- Lithops v2.5.9.dev0
2022-01-07 15:34:30,114 [DEBUG] lithops.config -- Config file not found
2022-01-07 15:34:30,115 [DEBUG] lithops.config -- Loading compute backend module: localhost
2022-01-07 15:34:30,115 [DEBUG] lithops.config -- Loading Storage backend module: localhost
2022-01-07 15:34:30,116 [DEBUG] lithops.storage.backends.localhost.localhost -- Creating Localhost storage client
2022-01-07 15:34:30,116 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-01-07 15:34:30,117 [DEBUG] lithops.localhost.localhost -- Creating Localhost compute client
2022-01-07 15:34:30,117 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-01-07 15:34:30,118 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 - Invoker initialized. Max workers: 1
2022-01-07 15:34:30,118 [DEBUG] lithops.executors -- Function executor for localhost created with ID: 1f1011-74
2022-01-07 15:34:30,119 [INFO] lit

2022-01-07 15:34:35,563 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A002 - Putting job into localhost queue
2022-01-07 15:34:35,564 [DEBUG] lithops.localhost.localhost -- Staring localhost job manager
2022-01-07 15:34:35,565 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 | JobID A002 - Job invoked (0.002s) - Activation ID: 1f1011-74-A002
2022-01-07 15:34:35,566 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A002 - Running 1 activations in the localhost worker
2022-01-07 15:34:35,566 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A002 - View execution logs at /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/lithops/logs/1f1011-74-A002.log
2022-01-07 15:34:35,581 [DEBUG] lithops.monitor -- ExecutorID 1f1011-74 - Starting Storage job monitor
2022-01-07 15:34:35,581 [INFO] lithops.wait -- ExecutorID 1f1011-74 - Getting results from functions
2022-01-07 15:34:36,808 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | 

2022-01-07 15:34:44,754 [DEBUG] lithops.job.serialize -- Modules to transmit: None
2022-01-07 15:34:44,754 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A005 - Function and modules found in local cache
2022-01-07 15:34:44,755 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A005 - Uploading data to the storage backend
2022-01-07 15:34:44,756 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A005 - Starting function invocation: inverted_map() - Total: 1 activations
2022-01-07 15:34:44,756 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 | JobID A005 - Worker processes: 16 - Chunksize: 16
2022-01-07 15:34:44,757 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A005 - Putting job into localhost queue
2022-01-07 15:34:44,758 [DEBUG] lithops.localhost.localhost -- Staring localhost job manager
2022-01-07 15:34:44,759 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 | JobID A005 - Job invoked (0.003s) - Activation ID: 1f1011-74-A005
2022-01-07 15:34:

2022-01-07 15:34:52,924 [DEBUG] lithops.executors -- ExecutorID 1f1011-74 - Finished getting results
2022-01-07 15:34:52,928 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A008 - Selected Runtime: python 
2022-01-07 15:34:52,929 [DEBUG] lithops.storage.storage -- Runtime metadata found in local memory cache
2022-01-07 15:34:52,930 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A008 - Serializing function and data
2022-01-07 15:34:52,934 [DEBUG] lithops.job.serialize -- Referenced modules: /Users/ah11500/opt/anaconda3/envs/yb1/lib/python3.8/site-packages/pandas/__init__.py
2022-01-07 15:34:52,935 [DEBUG] lithops.job.serialize -- Modules to transmit: None
2022-01-07 15:34:52,935 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A008 - Function and modules found in local cache
2022-01-07 15:34:52,935 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A008 - Uploading data to the storage backend
2022-01-07 15:34:52,937 [INFO] lithops.invokers -- ExecutorID 1f10

2022-01-07 15:35:00,081 [DEBUG] lithops.monitor -- ExecutorID 1f1011-74 - Pending: 0 - Running: 0 - Done: 1
2022-01-07 15:35:00,081 [DEBUG] lithops.monitor -- ExecutorID 1f1011-74 - Storage job monitor finished
2022-01-07 15:35:01,078 [DEBUG] lithops.future -- ExecutorID 1f1011-74 | JobID A010 - Got status from call 00000 - Activation ID: 1b174bb0d73a - Time: 0.28 seconds
2022-01-07 15:35:01,080 [DEBUG] lithops.future -- ExecutorID 1f1011-74 | JobID A010 - Got output from call 00000 - Activation ID: 1b174bb0d73a
2022-01-07 15:35:01,081 [INFO] lithops.executors -- ExecutorID 1f1011-74 - Cleaning temporary data
2022-01-07 15:35:01,094 [DEBUG] lithops.executors -- ExecutorID 1f1011-74 - Finished getting results
2022-01-07 15:35:01,097 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A011 - Selected Runtime: python 
2022-01-07 15:35:01,098 [DEBUG] lithops.storage.storage -- Runtime metadata found in local memory cache
2022-01-07 15:35:01,099 [DEBUG] lithops.job.job -- ExecutorID 1f1

...

2022-01-07 15:35:16,421 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 | JobID A016 - Worker processes: 16 - Chunksize: 16
2022-01-07 15:35:16,421 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A016 - Putting job into localhost queue
2022-01-07 15:35:16,423 [DEBUG] lithops.localhost.localhost -- Staring localhost job manager
2022-01-07 15:35:16,424 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 | JobID A016 - Job invoked (0.003s) - Activation ID: 1f1011-74-A016
2022-01-07 15:35:16,425 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A016 - Running 1 activations in the localhost worker
2022-01-07 15:35:16,425 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A016 - View execution logs at /private/var/folders/lg/pbb4ypp52kx1shc2q76nnnlc0000gn/T/lithops/logs/1f1011-74-A016.log
2022-01-07 15:35:16,438 [DEBUG] lithops.monitor -- ExecutorID 1f1011-74 - Starting Storage job monitor
2022-01-07 15:35:16,439 [INFO] lithops.wait -- ExecutorID 1f10

2022-01-07 15:35:25,594 [DEBUG] lithops.job.serialize -- Referenced modules: /Users/ah11500/opt/anaconda3/envs/yb1/lib/python3.8/site-packages/pandas/__init__.py
2022-01-07 15:35:25,595 [DEBUG] lithops.job.serialize -- Modules to transmit: None
2022-01-07 15:35:25,595 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A019 - Function and modules found in local cache
2022-01-07 15:35:25,596 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A019 - Uploading data to the storage backend
2022-01-07 15:35:25,598 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A019 - Starting function invocation: inverted_map() - Total: 1 activations
2022-01-07 15:35:25,598 [DEBUG] lithops.invokers -- ExecutorID 1f1011-74 | JobID A019 - Worker processes: 16 - Chunksize: 16
2022-01-07 15:35:25,599 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A019 - Putting job into localhost queue
2022-01-07 15:35:25,600 [DEBUG] lithops.localhost.localhost -- Staring localhost job mana

2022-01-07 15:35:34,772 [DEBUG] lithops.executors -- ExecutorID 1f1011-74 - Finished getting results
2022-01-07 15:35:34,774 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A022 - Selected Runtime: python 
2022-01-07 15:35:34,776 [DEBUG] lithops.storage.storage -- Runtime metadata found in local memory cache
2022-01-07 15:35:34,776 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A022 - Serializing function and data
2022-01-07 15:35:34,779 [DEBUG] lithops.job.serialize -- Referenced modules: None
2022-01-07 15:35:34,780 [DEBUG] lithops.job.serialize -- Modules to transmit: None
2022-01-07 15:35:34,781 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A022 - Function and modules found in local cache
2022-01-07 15:35:34,782 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A022 - Uploading data to the storage backend
2022-01-07 15:35:34,783 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A022 - Starting function invocation: inverted_reduce() - Total: 1 

2022-01-07 15:35:43,917 [DEBUG] lithops.future -- ExecutorID 1f1011-74 | JobID A024 - Got status from call 00000 - Activation ID: 3b5e2e03146a - Time: 0.01 seconds
2022-01-07 15:35:43,918 [DEBUG] lithops.future -- ExecutorID 1f1011-74 | JobID A024 - Got output from call 00000 - Activation ID: 3b5e2e03146a
2022-01-07 15:35:43,920 [INFO] lithops.executors -- ExecutorID 1f1011-74 - Cleaning temporary data
2022-01-07 15:35:43,934 [DEBUG] lithops.executors -- ExecutorID 1f1011-74 - Finished getting results
2022-01-07 15:35:43,938 [INFO] lithops.invokers -- ExecutorID 1f1011-74 | JobID A025 - Selected Runtime: python 
2022-01-07 15:35:43,939 [DEBUG] lithops.storage.storage -- Runtime metadata found in local memory cache
2022-01-07 15:35:43,940 [DEBUG] lithops.job.job -- ExecutorID 1f1011-74 | JobID A025 - Serializing function and data
2022-01-07 15:35:43,942 [DEBUG] lithops.job.serialize -- Referenced modules: None
2022-01-07 15:35:43,943 [DEBUG] lithops.job.serialize -- Modules to transmit:

2022-01-07 15:35:51,041 [DEBUG] lithops.localhost.localhost -- ExecutorID 1f1011-74 | JobID A027 - Execution finished
2022-01-07 15:35:51,042 [DEBUG] lithops.localhost.localhost -- Localhost job manager stopped
2022-01-07 15:35:52,104 [DEBUG] lithops.monitor -- ExecutorID 1f1011-74 - Pending: 0 - Running: 0 - Done: 1
2022-01-07 15:35:52,105 [DEBUG] lithops.monitor -- ExecutorID 1f1011-74 - Storage job monitor finished
2022-01-07 15:35:52,105 [DEBUG] lithops.future -- ExecutorID 1f1011-74 | JobID A027 - Got status from call 00000 - Activation ID: 19d1a085d31f - Time: 0.01 seconds
2022-01-07 15:35:52,106 [DEBUG] lithops.future -- ExecutorID 1f1011-74 | JobID A027 - Got output from call 00000 - Activation ID: 19d1a085d31f
2022-01-07 15:35:52,108 [INFO] lithops.executors -- ExecutorID 1f1011-74 - Cleaning temporary data
2022-01-07 15:35:52,121 [DEBUG] lithops.executors -- ExecutorID 1f1011-74 - Finished getting results


# Question 3
## Shuffle

MapReduceServerlessEngine deploys both map and reduce tasks as serverless invocations.   
However, once map stage completed, the result are transferred from the map tasks to the SQLite database located on the client machine (laptop in your case), then performed local shuffle and then invoked reduce tasks passing them relevant parameters.

(To support your answers, feel free to use examples, Images, etc.)
<br><br>

**1. Explain why this approach is not efficient and what are cons and pros of such architecture in general. In broader scope you may assume that MapReduceServerlessEngine executed in some powerful machine and not just laptop.**

\<your answer here>

<br><br>
**2. Suggest how can you improve shuffle so intermediate data will not be downloaded to the client at all and shuffle performed in the cloud as well. Explain pros and cons of the approaches you suggest.**


\<your answer here>

<br><br>
**3. Can you make serverless shuffle?**


\<your answer here>

<br><br><br><br>
Good Luck :) 